In [1]:
import gurobipy as gp
from gurobipy import GRB
from models.JGS import JGSModel
import pandas as pd
import os

In [2]:
def retrieve_format_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        first_row = lines[0].strip().split()
        num_jobs = int(first_row[0])
        num_tools = int(first_row[1])
        magazine_capacity = int(first_row[2])
        
        job_tools = {job: [] for job in range(1, num_jobs + 1)}
        
        for tool_index, line in enumerate(lines[1:], start=1):
            tool_requirements = [int(x) for x in line.strip().split()]
            for job_index, requirement in enumerate(tool_requirements, start=1):
                if requirement == 1:
                    job_tools[job_index].append(tool_index)
                    
    return num_jobs, num_tools, magazine_capacity, job_tools

In [18]:
file_path = '../data/MTSP/Laporte/Tabela4/L1-1.txt'
num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)

print("Number of Jobs:", num_jobs)
print("Number of Tools:", num_tools)
print("Magazine Capacity:", magazine_capacity)
print("Job Tool Requirements:", job_tools)

Number of Jobs: 9
Number of Tools: 15
Magazine Capacity: 5
Job Tool Requirements: {1: [10, 12], 2: [3, 8, 12, 13, 14], 3: [4, 10], 4: [6, 8, 9, 14, 15], 5: [4, 9, 15], 6: [2, 5, 7, 13, 15], 7: [11, 14], 8: [4, 5, 14], 9: [1, 9]}


In [ ]:
jobs = [x for x in range(1, num_jobs + 1)]
tools = [x for x in range(1, num_tools + 1)]
model = JGSModel(jobs, tools, magazine_capacity, job_tools)

model.solve()

solution = model.get_solution()
print(solution)

In [3]:
directory = '../data/MTSP/Laporte/Tabela4/'
data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
        data.append([num_jobs, num_tools, magazine_capacity, job_tools])

df = pd.DataFrame(data, columns=['num_jobs', 'num_tools', 'magazine_capacity', 'job_tools'])
df = df.sample(frac=0.02).reset_index(drop=True)

In [4]:
from models.JGS import JGSModel
results = []

for index, row in df.iterrows():
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    model = JGSModel(jobs, tools, magazine_capacity, job_tools)
    model.model.setParam('OutputFlag', 0)
    model.model.setParam('TimeLimit', 60*60)

    model.solve()

    if model.model.status == GRB.OPTIMAL:
        solution_found = True
        solution = model.get_solution()
    else:
        solution_found = False
        solution = None

    results.append([solution_found, solution])

results_df = pd.DataFrame(results, columns=['solution_found', 'solution'])
results_df
        



Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-23


KeyError: (1, 0)

In [5]:
results_df.solution

0    ({(1, 8): 1.0, (2, 1): 1.0, (3, 9): 1.0, (4, 7...
1    ({(1, 1): 1.0, (2, 1): 1.0, (3, 1): 1.0, (4, 1...
2    ({(1, 3): 1.0, (2, 9): 1.0, (3, 2): 1.0, (4, 6...
3    ({(1, 3): 1.0, (2, 5): 1.0, (3, 1): 1.0, (4, 4...
4    ({(1, 2): 1.0, (2, 8): 1.0, (3, 3): 1.0, (4, 5...
5    ({(1, 2): 1.0, (2, 7): 1.0, (3, 2): 1.0, (4, 5...
6    ({(1, 2): 1.0, (2, 3): 1.0, (3, 9): 1.0, (4, 8...
Name: solution, dtype: object